In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

In [4]:
#creating dataset
path = os.getcwd()
path = os.path.join(path,'images\P')

categories = ['cracks', 'nocracks']
#appending 500 images with cracks
pathcracks = os.path.join(path,'crackstrain')
dataset = []
label = []
files =  os.listdir(pathcracks)[0:1000]
for f in files:
    imagepath = os.path.join(pathcracks,f)
    img = cv2.imread(imagepath)
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    dataset.append(img)
    label.append(1)
print(f)

#now loading 1500 non crack images
pathnocracks = os.path.join(path,'nocrackstrain')
files =  os.listdir(pathnocracks)[0:1000]
for f in files:
    imagepath = os.path.join(pathnocracks,f)
    img = cv2.imread(imagepath)
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    dataset.append(img)
    label.append(0)
print(f)
final_dataset = np.array(dataset)
final_label = np.array(label)
final_dataset.shape, final_label.shape
print(final_label.shape)

066-97.jpg
005-45.jpg
(2000,)


In [5]:
print(final_dataset.shape)
#its time to do some machine learning
import tensorflow as tf


(2000, 256, 256, 3)


In [6]:
#preprocessing
final_dataset = final_dataset/255
final_label = tf.one_hot(final_label,2)

In [7]:
print(final_dataset)
print(final_label)

[[[[0.36078431 0.39215686 0.44313725]
   [0.37647059 0.40784314 0.45882353]
   [0.41176471 0.44313725 0.49411765]
   ...
   [0.46666667 0.50196078 0.55686275]
   [0.44705882 0.48235294 0.5372549 ]
   [0.42352941 0.45882353 0.51372549]]

  [[0.3372549  0.36862745 0.41960784]
   [0.35294118 0.38431373 0.43529412]
   [0.38823529 0.41960784 0.47058824]
   ...
   [0.47058824 0.50588235 0.56078431]
   [0.47058824 0.50588235 0.56078431]
   [0.46666667 0.50196078 0.55686275]]

  [[0.37647059 0.40784314 0.45882353]
   [0.36862745 0.4        0.45098039]
   [0.37647059 0.40784314 0.45882353]
   ...
   [0.41568627 0.45098039 0.50588235]
   [0.42352941 0.45882353 0.51372549]
   [0.43921569 0.4745098  0.52941176]]

  ...

  [[0.45098039 0.48627451 0.54117647]
   [0.42745098 0.4627451  0.51764706]
   [0.38039216 0.41568627 0.47058824]
   ...
   [0.34117647 0.37647059 0.42745098]
   [0.37254902 0.40784314 0.45882353]
   [0.38823529 0.42352941 0.4745098 ]]

  [[0.35686275 0.39215686 0.44705882]
   [0.4

In [10]:
#creating placeholders
def create_placeholder(h,w,c,y):
    X= tf.placeholder(tf.float32, shape= [None,h,w,c])
    Y= tf.placeholder(tf.float32, shape= [None,y])
    return X,Y

In [11]:
#testing placeholder
x,y = create_placeholder(256,256,3,2)
print(str(x)+" " + str(y))


Tensor("Placeholder:0", shape=(?, 256, 256, 3), dtype=float32) Tensor("Placeholder_1:0", shape=(?, 2), dtype=float32)


In [12]:
#time to initialize the filters 
def initialize_parameters():
    tf.set_random_seed(1)
    W1 = tf.get_variable('W1', [8,8,3,4], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W2 = tf.get_variable('W2', [4,4,4,8], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W3 = tf.get_variable('W3', [2,2,8,16], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    parameters = {
        'W1':W1,
        'W2':W2,
        'W3':W3
    }
    return parameters

In [13]:
tf.reset_default_graph()
with tf.Session() as test:
    parameters = initialize_parameters()
    b = tf.global_variables_initializer()
    test.run(b)
    print("W1 = " + str(parameters["W1"].eval()[1,1,1]))
    print("W2 = " + str(parameters["W2"].eval()[1,1,1]))
    print("W3 = " + str(parameters["W3"].eval()[1,1,1]))


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
W1 = [-0.04939678  0.07564741  0.0370781  -0.06331497]
W2 = [ 0.08549185 -0.16564402 -0.00140636 -0.1667098  -0.01183425  0.03997822
  0.04054093  0.02668823]
W3 = [-0.08566415  0.17750949  0.11974221  0.16773748 -0.0830943  -0.08058
 -0.00577033 -0.14643836  0.24162132 -0.05857408 -0.19055021  0.1345228
 -0.22779644 -0.1601823  -0.16117483 -0.10286498]


In [30]:
def forward_propagation(X,parameters):
    W1 = parameters['W1']
    W2 = parameters['W2']
    W3 = parameters['W3']
    Z1 = tf.nn.conv2d(X,W1,strides = [1,1,1,1],padding = 'SAME')
    A1 = tf.nn.relu(Z1)
    P1 = tf.nn.max_pool(A1,ksize = [1,8,8,1], strides= [1,8,8,1], padding = 'SAME')
    Z2 = tf.nn.conv2d(P1,W2,strides = [1,1,1,1],padding = 'SAME')
    A2 = tf.nn.relu(Z2)
    P2 = tf.nn.max_pool(A2,ksize = [1,4,4,1], strides= [1,4,4,1], padding = 'SAME')
    Z3 = tf.nn.conv2d(P2,W3,strides = [1,1,1,1],padding = 'SAME')
    A3 = tf.nn.relu(Z3)
    P3 = tf.nn.max_pool(A3,ksize = [1,4,4,1], strides= [1,4,4,1], padding = 'SAME')
    P3 = tf.contrib.layers.flatten(P3)
    Z4 = tf.contrib.layers.fully_connected(P3, 2, activation_fn=None)
    print(P3.shape)
    return Z4

In [32]:
tf.reset_default_graph()

with tf.Session() as sess:
    np.random.seed(1)
    X, Y = create_placeholder(256, 256, 3, 2)
    parameters = initialize_parameters()
    Z4 = forward_propagation(X, parameters)
    init = tf.global_variables_initializer()
    sess.run(init)
    a = sess.run(Z4, {X: np.random.randn(1,256,256,3), Y: np.random.randn(1,2)})
    print("Z4 = " + str(a))

(?, 64)
Z4 = [[ 0.9680337 -1.1696649]]


In [35]:
def compute_cost(Z4,Y):
    cost = tf.nn.softmax_cross_entropy_with_logits(logits = Z4, labels=Y)
    cost = tf.reduce_mean(cost)
    return cost

In [38]:

tf.reset_default_graph()

with tf.Session() as sess:
    np.random.seed(1)
    X, Y = create_placeholder(256, 256, 3, 2)
    parameters = initialize_parameters()
    Z4 = forward_propagation(X, parameters)
    init = tf.global_variables_initializer()
    sess.run(init)
    
    
    cost = compute_cost(Z4,Y)
    a = sess.run(cost, {X: np.random.randn(500,256,256,3), Y: np.random.randn(500,2)})
    print(a)

(?, 64)


ResourceExhaustedError: OOM when allocating tensor with shape[500,3,257,257] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node Conv2D (defined at <ipython-input-30-a385e5953658>:5) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[node Mean (defined at <ipython-input-35-c1595742a5cd>:3) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'Conv2D', defined at:
  File "C:\Users\NEON\Anaconda3\envs\PythonGPU\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\NEON\Anaconda3\envs\PythonGPU\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\NEON\Anaconda3\envs\PythonGPU\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_i